In [9]:
%logstop
%logstart -rtq ~/.logs/dw.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `pw` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a pandas `DataFrame` by making use of the `pd.read_csv` function to parse the data into a `DataFrame`. You may want to check the pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [`pw` miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np
import gzip

In [5]:
# load the 2017 data

with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [6]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']

with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f, names =  col_names)

practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [7]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)

chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `pw` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `pw` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `pw` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: `[('act_cost', (total, mean, std, q25, median, q75)), ...]`

In [80]:
summary = pd.DataFrame(scripts.describe())
summary

,items,nic,act_cost,quantity
count,973193.000000,973193.000000,973193.000000,973193.000000
mean,9.133136,73.058915,67.986613,741.329835
std,29.204198,188.070257,174.401703,3665.426958
min,1.000000,0.000000,0.040000,0.000000
25%,1.000000,7.800000,7.330000,28.000000
50%,2.000000,22.640000,21.220000,100.000000
75%,6.000000,65.000000,60.670000,350.000000
max,2384.000000,16320.000000,15108.320000,577720.000000


In [81]:
stats = list(summary.index)[0:3]
stats.extend(list(summary.index)[4:7])
stats

['count', 'mean', 'std', '25%', '50%', '75%']

In [82]:
summary_stats = [(col, (summary[col][stats[0]] * summary[col][stats[1]],summary[col][stats[1]],summary[col][stats[2]],
        summary[col][stats[3]],summary[col][stats[4]],summary[col][stats[5]])) 
 for col in [list(summary.columns)[0],list(summary.columns)[3],list(summary.columns)[1],list(summary.columns)[2]]]

In [83]:
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0)),
 ('nic',
  (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999998,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67))]

In [84]:
grader.score.dw__summary_statistics(summary_stats)

Your score: 1.000


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [85]:
comman_item = tuple(scripts.groupby('bnf_name')['items'].agg(np.sum).reset_index().sort_values(by = 'items',
                                                                          ascending = False).iloc[0,:2])

In [86]:
most_common_item = [comman_item]

In [87]:
grader.score.dw__most_common_item(most_common_item)

Your score: 1.000


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the `pw` miniproject).

In [10]:
srt = practices.sort_values('post_code').drop_duplicates('code', keep='first')
srt.head()

,code,name,addr_1,addr_2,borough,village,post_code
1896,E82060,PARKBURY HOUSE SURGERY,PARKBURY HOUSE,ST.PETERS STREET,ST.ALBANS,HERTFORDSHIRE,AL1 3HD
1871,E82031,MALTINGS SURGERY,THE MALTINGS SURGERY,8-14 VICTORIA STREET,ST ALBANS,HERTFORDSHIRE,AL1 3JB
1849,E82004,HATFIELD ROAD SURGERY,61 HATFIELD ROAD,NaN,ST.ALBANS,HERTFORDSHIRE,AL1 4JE
1848,E82002,WRAFTON HOUSE SURGERY,WRAFTON HOUSE SURGERY,9/11 WELLFIELD ROAD,HATFIELD,HERTFORDSHIRE,AL10 0BS
9812,Y05146,HCT LYMPHOEDEMA AT WEST ESSEX CCG,QUEENSWAY HEALTH CENTRE,QUEENSWAY,HATFIELD,HERTFORDSHIRE,AL10 0LF


In [11]:
merged = scripts.merge(srt, left_on='practice', right_on='code')

merged.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [12]:
total_items_by_postcode = merged.groupby(['post_code','bnf_name'])[['items']].sum()
total_items_by_postcode.head()

items
post_code bnf_name                                       
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7
          3m Health Care_Cavilon No Sting Barrier       2
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2
          Abidec_Dps                                   63
          Able Spacer + Sml/Med Mask                    1

In [13]:
total_items_by_postcode.reset_index(inplace=True)
total_items_by_postcode.head()

,post_code,bnf_name,items
0,B11 4BW,3m Health Care_Cavilon Durable Barrier C,7
1,B11 4BW,3m Health Care_Cavilon No Sting Barrier,2
2,B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2
3,B11 4BW,Abidec_Dps,63
4,B11 4BW,Able Spacer + Sml/Med Mask,1


In [14]:
total_items_max = total_items_by_postcode.groupby('post_code')[['items']].max()
total_items_max.head()

,items
post_code,
B11 4BW,706
B12 9LP,425
B18 7AL,556
B21 9RY,1033
B23 6DJ,599


In [16]:
total_items = total_items_max.merge(total_items_by_postcode, on=['post_code','items'], how = 'left')
total_items.head()

,post_code,items,bnf_name
0,B11 4BW,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,425,Paracet_Tab 500mg
2,B18 7AL,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,599,Lansoprazole_Cap 30mg (E/C Gran)


In [17]:
total_items_sorted = total_items.sort_values('bnf_name').drop_duplicates(['post_code','items'], keep='first')
total_items_sorted.head()

,post_code,items,bnf_name
73,E15 4ES,451,Amlodipine_Tab 10mg
66,DN16 2AB,1072,Amlodipine_Tab 5mg
255,WS9 8AJ,396,Amlodipine_Tab 5mg
142,NG2 7SD,605,Amlodipine_Tab 5mg
97,KT16 8HZ,401,Amlodipine_Tab 5mg


In [19]:
s = total_items_by_postcode.groupby('post_code')[['items']].sum().reset_index()
s.head()

,post_code,items
0,B11 4BW,22731
1,B12 9LP,17073
2,B18 7AL,20508
3,B21 9RY,31027
4,B23 6DJ,28011


In [20]:
final_df = s.merge(total_items_sorted, on='post_code')
final_df.head()

,post_code,items_x,items_y,bnf_name
0,B11 4BW,22731,706,Salbutamol_Inha 100mcg (200 D) CFF
1,B12 9LP,17073,425,Paracet_Tab 500mg
2,B18 7AL,20508,556,Salbutamol_Inha 100mcg (200 D) CFF
3,B21 9RY,31027,1033,Metformin HCl_Tab 500mg
4,B23 6DJ,28011,599,Lansoprazole_Cap 30mg (E/C Gran)


In [22]:
final_df['amt%'] = final_df['items_y'].astype(float)/final_df['items_x'].astype(float)
final_df.head()

,post_code,items_x,items_y,bnf_name,amt%
0,B11 4BW,22731,706,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,17073,425,Paracet_Tab 500mg,0.024893
2,B18 7AL,20508,556,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,31027,1033,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,28011,599,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [23]:
result = final_df[['post_code','bnf_name','amt%']]
result.head()

,post_code,bnf_name,amt%
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,Paracet_Tab 500mg,0.024893
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [31]:
result = result.head(100)

In [43]:
items_by_region = tuple(result.values.tolist())

In [46]:
items_by_region[:5]

(['B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346],
 ['B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788],
 ['B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472],
 ['B21 9RY', 'Metformin HCl_Tab 500mg', 0.03329358300834757],
 ['B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.021384456106529576])

In [45]:
grader.score.dw__items_by_region(items_by_region)

Your score: 1.000


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [8]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [9]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [13]:
mask = chem['NAME'].str.contains('|'.join(opioids), case=False)
opioid_codes = chem[mask]['CHEM SUB']
scripts['opioids'] = scripts['bnf_code'].isin(opioid_codes).astype(int)

opioids_per_practice = scripts.groupby('practice')['opioids'].mean()

opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: opioids, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [14]:
relative_opioids_per_practice = opioids_per_practice - scripts['opioids'].mean()
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: opioids, dtype: float64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [15]:
standard_error_per_practice = scripts['opioids'].std() / np.sqrt(scripts['practice'].value_counts())
opioid_scores = relative_opioids_per_practice / standard_error_per_practice

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_code, practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [16]:
top_opioids = opioid_scores.sort_values(ascending=False)[:100]
top_opioids = pd.DataFrame(top_opioids.rename('z_score')).reset_index()
top_opioids.columns = ['practice', 'z_score']

In [17]:
top_opioids

,practice,z_score
0,Y01852,11.695818
1,Y03006,7.339043
2,Y03668,6.150582
3,G81703,5.123032
4,Y04997,4.958866
...,...,...
95,D81024,1.696924
96,A81033,1.694832
97,P81684,1.691316
98,N81121,1.686184


In [18]:
unique_practices = practices.groupby('code')['name'].min()
unique_practices = pd.DataFrame(unique_practices)

In [19]:
results = top_opioids.merge(unique_practices,
                           how='left',
                           left_on='practice',
                           right_index=True)

In [20]:
results = results.merge(pd.DataFrame(scripts.groupby('practice').size().rename('n_scripts')),
                       how='left',
                       left_on='practice',
                       right_index=True)

In [21]:
results = results[['practice', 'name', 'z_score', 'n_scripts']]

In [22]:
anomalies = list(tuple(zip(results['name'], results['z_score'], results['n_scripts'])))

In [23]:
anomalies = [tuple(r) for r in results.values]

In [24]:
grader.score.dw__script_anomalies(anomalies)

Your score: 1.000


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the growth rate in prescription rate from June 2016 to January 2017 for each `bnf_name`. The growth is defined as

$$
\text{growth rate} = \frac{x_1 - x_0}{x_0},
$$
where $x_1$ is the current value and $x_0$ is the previous value.


We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative growth rate) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [25]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz',compression='gzip')

In [26]:
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [27]:
raw_2017_count = scripts.groupby(['bnf_name'])['items'].count().rename('raw_2017_count')
raw_2016_count = scripts16.groupby(['bnf_name'])['items'].count().rename('raw_2016_count')

In [28]:
growth = ((raw_2017_count - raw_2016_count)/raw_2016_count).reset_index(name='growth_rate')
growth_df = growth.merge(raw_2016_count.reset_index(), on ='bnf_name')
growth_df.fillna(0, inplace=True)
growth_df.head()

,bnf_name,growth_rate,raw_2016_count
0,365 Film 15cm x 20cm VP Adh Film Dress,0.0,1
1,365 Film 4cm x 5cm VP Adh Film Dress,0.0,1
2,365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.0,3
3,365 Non Woven Island 10cm x 10cm Adh Dre,0.0,1
4,365 Non Woven Island 10cm x 15cm Adh Dre,0.0,1


In [29]:
growth_df = growth_df[growth_df['raw_2016_count'] >= 50]
growth_df.head()

,bnf_name,growth_rate,raw_2016_count
10,3m Health Care_Cavilon Durable Barrier C,-0.010909,825
11,3m Health Care_Cavilon No Sting 1ml Barr,-0.034632,231
12,3m Health Care_Cavilon No Sting 3ml Barr,-0.104762,105
13,3m Health Care_Cavilon No Sting Barrier,-0.092511,454
16,A.S Saliva Orthana Spy 50ml (App),0.440476,84


In [30]:
growth_df.sort_values(by='growth_rate', ascending=False, inplace=True)
growth_df.head()

,bnf_name,growth_rate,raw_2016_count
1745,Butec_Transdermal Patch 5mcg/hr,3.467742,62
1743,Butec_Transdermal Patch 10mcg/hr,3.000000,69
5114,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86
9907,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193
11710,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52


In [31]:
script_growth = list(pd.concat([growth_df.head(50), growth_df.tail(50)]).itertuples(index=False, name=None))
script_growth[:3]

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86)]

In [32]:
grader.score.dw__script_growth(script_growth)

Your score: 1.000


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [33]:
unique_bnf_code = scripts.bnf_code.nunique()
p = 1/unique_bnf_code
rates = scripts.bnf_code.value_counts()/len(scripts)
rare = rates < .1 * p

rare_codes = rates[rare].index.unique()
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [34]:
total_cost_per_practice = scripts.groupby('practice')['act_cost'].sum().reset_index(name='total_cost_per_practice')
total_cost_per_practice.iloc[:3]

,practice,total_cost_per_practice
0,A81005,103840.82
1,A81007,113482.49
2,A81011,159507.03


In [36]:
rare_cost_per_practice = (scripts[scripts['rare']==True].groupby(
    'practice')['act_cost'].sum().reset_index(name = 'rare_cost_per_practice'))
script_costs = total_cost_per_practice.merge(rare_cost_per_practice, on='practice', how='left')
script_costs.iloc[:3]

,practice,total_cost_per_practice,rare_cost_per_practice
0,A81005,103840.82,1247.83
1,A81007,113482.49,951.06
2,A81011,159507.03,816.02


In [37]:
script_costs['rare_cost_prop'] =script_costs['rare_cost_per_practice'] / script_costs['total_cost_per_practice']
script_costs.iloc[:3]

,practice,total_cost_per_practice,rare_cost_per_practice,rare_cost_prop
0,A81005,103840.82,1247.83,0.012017
1,A81007,113482.49,951.06,0.008381
2,A81011,159507.03,816.02,0.005116


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [38]:
script_costs['relative_rare_cost_prop'] = (script_costs['rare_cost_prop'] - scripts[scripts['rare']==True]['act_cost'].sum() / scripts['act_cost'].sum())

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [39]:
script_costs['standard_errors'] = script_costs['relative_rare_cost_prop'].std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(practice_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [40]:
script_costs['z_scores'] = (script_costs['relative_rare_cost_prop'] / script_costs['standard_errors'])
script_costs.fillna(0, inplace=True)
script_costs.iloc[:3]

,practice,total_cost_per_practice,rare_cost_per_practice,rare_cost_prop,relative_rare_cost_prop,standard_errors,z_scores
0,A81005,103840.82,1247.83,0.012017,-0.003946,0.063534,-0.062111
1,A81007,113482.49,951.06,0.008381,-0.007582,0.063534,-0.119342
2,A81011,159507.03,816.02,0.005116,-0.010847,0.063534,-0.170728


In [41]:
unique_practices = practices.sort_values('name').groupby('code', sort=False).first()
unique_practices.reset_index(inplace=True)
final_scripts = script_costs.merge(
    unique_practices[['name', 'code']], how = 'inner', left_on = 'practice', right_on = 'code', sort = False)

In [43]:
final_scripts.iloc[:3]

,practice,total_cost_per_practice,rare_cost_per_practice,rare_cost_prop,relative_rare_cost_prop,standard_errors,z_scores,name,code
0,A81005,103840.82,1247.83,0.012017,-0.003946,0.063534,-0.062111,SPRINGWOOD SURGERY,A81005
1,A81007,113482.49,951.06,0.008381,-0.007582,0.063534,-0.119342,BANKHOUSE SURGERY,A81007
2,A81011,159507.03,816.02,0.005116,-0.010847,0.063534,-0.170728,CHADWICK PRACTICE,A81011


In [44]:
final_scripts = final_scripts.sort_values(by='name', ascending=True)
rare_scripts = [(row['practice'], row['name'], row['z_scores']) for index, row in final_scripts.iterrows()]

In [45]:
rare_scripts = sorted(rare_scripts, key=lambda x: x[2], reverse = True)[:100]
rare_scripts[:3]

[('Y03472', 'CONSULTANT DIABETES TEAM', 15.488406625215609),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 14.408360263244049),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.183051624616913)]

In [46]:
grader.score.dw__rare_scripts(rare_scripts)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*